In [1]:
from pyspark.sql import *
from pyspark.sql.types import *

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
1,application_1619107925254_0005,pyspark3,idle,Link,Link,✔


SparkSession available as 'spark'.
/usr/bin/anaconda/envs/py35/lib/python3.5/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')
/usr/bin/anaconda/envs/py35/lib/python3.5/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')

In [25]:
spark_df = spark.read.csv('wasb:///inputData/worldometer_data.csv', header=True, inferSchema=True)

In [26]:
spark_df.describe()
spark_df.show()

+--------------+-------------+----------+----------+--------+-----------+---------+--------------+------------+-----------+----------------+----------------+-------------+----------+------------+--------------------+
|Country/Region|    Continent|Population|TotalCases|NewCases|TotalDeaths|NewDeaths|TotalRecovered|NewRecovered|ActiveCases|Serious,Critical|Tot Cases/1M pop|Deaths/1M pop|TotalTests|Tests/1M pop|          WHO Region|
+--------------+-------------+----------+----------+--------+-----------+---------+--------------+------------+-----------+----------------+----------------+-------------+----------+------------+--------------------+
|           USA|North America| 331198130|   5032179|    null|     162804|     null|       2576668|        null|    2292707|           18296|           15194|        492.0|  63139605|      190640|            Americas|
|        Brazil|South America| 212710692|   2917562|    null|      98644|     null|       2047660|        null|     771258|         

In [93]:
from pyspark.sql.functions import *
spark_df.select([count(when(col(c).isNull(), c)).alias(c) for c in spark_df.columns]).show()

+--------------+---------+----------+----------+--------+-----------+---------+--------------+------------+-----------+----------------+----------------+-------------+----------+------------+----------+
|Country/Region|Continent|Population|TotalCases|NewCases|TotalDeaths|NewDeaths|TotalRecovered|NewRecovered|ActiveCases|Serious,Critical|Tot Cases/1M pop|Deaths/1M pop|TotalTests|Tests/1M pop|WHO Region|
+--------------+---------+----------+----------+--------+-----------+---------+--------------+------------+-----------+----------------+----------------+-------------+----------+------------+----------+
|             0|        1|         1|         0|     205|         21|      206|             4|         206|          4|              87|               1|           22|        18|          18|        25|
+--------------+---------+----------+----------+--------+-----------+---------+--------------+------------+-----------+----------------+----------------+-------------+----------+----------

In [104]:
spark_newdf = spark_df.na.fill(0)

In [105]:
spark_newdf.select([count(when(col(c).isNull(), c)).alias(c) for c in spark_newdf.columns]).show()

+--------------+---------+----------+----------+--------+-----------+---------+--------------+------------+-----------+----------------+----------------+-------------+----------+------------+----------+
|Country/Region|Continent|Population|TotalCases|NewCases|TotalDeaths|NewDeaths|TotalRecovered|NewRecovered|ActiveCases|Serious,Critical|Tot Cases/1M pop|Deaths/1M pop|TotalTests|Tests/1M pop|WHO Region|
+--------------+---------+----------+----------+--------+-----------+---------+--------------+------------+-----------+----------------+----------------+-------------+----------+------------+----------+
|             0|        1|         0|         0|       0|          0|        0|             0|           0|          0|               0|               0|            0|         0|           0|        25|
+--------------+---------+----------+----------+--------+-----------+---------+--------------+------------+-----------+----------------+----------------+-------------+----------+----------

In [96]:
spark_newdf = spark_newdf.drop('NewCases', 'NewDeaths','NewRecovered','WHORegion','TotCases/1Mpop','Deaths/1Mpop','Tests/1Mpop','WHORegion')
spark_newdf.show()

+--------------+-------------+----------+----------+-----------+--------------+-----------+----------------+----------------+-------------+----------+------------+--------------------+
|Country/Region|    Continent|Population|TotalCases|TotalDeaths|TotalRecovered|ActiveCases|Serious,Critical|Tot Cases/1M pop|Deaths/1M pop|TotalTests|Tests/1M pop|          WHO Region|
+--------------+-------------+----------+----------+-----------+--------------+-----------+----------------+----------------+-------------+----------+------------+--------------------+
|           USA|North America| 331198130|   5032179|     162804|       2576668|    2292707|           18296|           15194|        492.0|  63139605|      190640|            Americas|
|        Brazil|South America| 212710692|   2917562|      98644|       2047660|     771258|            8318|           13716|        464.0|  13206188|       62085|            Americas|
|         India|         Asia|1381344997|   2025409|      41638|       1377

In [114]:
from functools import reduce
spark_finaldf = reduce(DataFrame.drop, ['NewCases', 'NewDeaths','NewRecovered','WHO Region',
                        'Tot Cases/1M pop','Deaths/1M pop','Tests/1M pop',
                        'WHORegion'], spark_newdf)
spark_finaldf.show()

+--------------+-------------+----------+----------+-----------+--------------+-----------+----------------+----------+
|Country/Region|    Continent|Population|TotalCases|TotalDeaths|TotalRecovered|ActiveCases|Serious,Critical|TotalTests|
+--------------+-------------+----------+----------+-----------+--------------+-----------+----------------+----------+
|           USA|North America| 331198130|   5032179|     162804|       2576668|    2292707|           18296|  63139605|
|        Brazil|South America| 212710692|   2917562|      98644|       2047660|     771258|            8318|  13206188|
|         India|         Asia|1381344997|   2025409|      41638|       1377384|     606387|            8944|  22149351|
|        Russia|       Europe| 145940924|    871894|      14606|        676357|     180931|            2300|  29716907|
|  South Africa|       Africa|  59381566|    538184|       9604|        387316|     141264|             539|   3149807|
|        Mexico|North America| 129066160

In [121]:
spark_finaldf = spark_finaldf.withColumnRenamed('Country/Region', 'Country')
spark_finaldf = spark_finaldf.withColumnRenamed('Serious,Critical', 'Critical')
spark_finaldf.show()

+------------+-------------+----------+----------+-----------+--------------+-----------+--------+----------+
|     Country|    Continent|Population|TotalCases|TotalDeaths|TotalRecovered|ActiveCases|Critical|TotalTests|
+------------+-------------+----------+----------+-----------+--------------+-----------+--------+----------+
|         USA|North America| 331198130|   5032179|     162804|       2576668|    2292707|   18296|  63139605|
|      Brazil|South America| 212710692|   2917562|      98644|       2047660|     771258|    8318|  13206188|
|       India|         Asia|1381344997|   2025409|      41638|       1377384|     606387|    8944|  22149351|
|      Russia|       Europe| 145940924|    871894|      14606|        676357|     180931|    2300|  29716907|
|South Africa|       Africa|  59381566|    538184|       9604|        387316|     141264|     539|   3149807|
|      Mexico|North America| 129066160|    462690|      50517|        308848|     103325|    3987|   1056915|
|        P

In [122]:
spark_finaldf.show()

+------------+-------------+----------+----------+-----------+--------------+-----------+--------+----------+
|     Country|    Continent|Population|TotalCases|TotalDeaths|TotalRecovered|ActiveCases|Critical|TotalTests|
+------------+-------------+----------+----------+-----------+--------------+-----------+--------+----------+
|         USA|North America| 331198130|   5032179|     162804|       2576668|    2292707|   18296|  63139605|
|      Brazil|South America| 212710692|   2917562|      98644|       2047660|     771258|    8318|  13206188|
|       India|         Asia|1381344997|   2025409|      41638|       1377384|     606387|    8944|  22149351|
|      Russia|       Europe| 145940924|    871894|      14606|        676357|     180931|    2300|  29716907|
|South Africa|       Africa|  59381566|    538184|       9604|        387316|     141264|     539|   3149807|
|      Mexico|North America| 129066160|    462690|      50517|        308848|     103325|    3987|   1056915|
|        P

In [123]:
spark_finaldf.write.saveAsTable("hvac1")

In [126]:
spark_finaldf.write.csv('wasb:///inputData/preprocessed/worldometer_data_final.csv')